In [ ]:
import sys

sys.path.append('../..')

# Step of the underlying

In [ ]:
step = 1 / 360.

# Max maturity of underlying

In [ ]:
maturity = 5.

# Data

In [ ]:
N = 20000

## Underlyings

In [ ]:
from maths.montecarlo.processes.brownianmotions import GeometricBrownianMotion

time_grid = GeometricBrownianMotion.generate_time_grid(0, maturity, step)

x_0 = [100]
drift = 0.12
vol = 0.2

underlyings = [GeometricBrownianMotion(time_grid, x_0, drift, vol) for _ in range(N)]

## Randomization variables

In [ ]:
from scipy.stats import expon
 
mean = maturity / 2.
exp_distrib = expon(loc=0, scale=mean)

print "Mean = %s"%mean

In [ ]:
from utils import v_time_offseter

In [ ]:
zetas = exp_distrib.rvs(size=N)

zetas = v_time_offseter(zetas, time_grid)

print zetas.shape

## Load intensities of obligors

In [ ]:
import dill

raw_intensities = None
with open('../../runs/intensities/intensities_no_I5_new.pkl', 'rb') as f:
    raw_intensities = dill.load(f)
    
raw_intensities.keys()

In [ ]:
import numpy as np

raw_idios = np.array(raw_intensities['idio'])
raw_groups = np.array(raw_intensities['groups'])

In [ ]:
import numpy

subsets = []
hazard_rates = []
pillars = []

for d_idio in raw_idios:
    subsets.append(d_idio['id'])
    hazard_rates.append(numpy.maximum(d_idio['hazard_rates'], 0.))
    pillars.append(d_idio['pillars'])
    
for d_groups in raw_groups:    
    subsets.append(d_groups['ids'])
    hazard_rates.append(numpy.maximum(d_groups['hazard_rates'], 0.))
    pillars.append(d_groups['pillars'])

In [ ]:
for s, hz, p in zip(subsets, hazard_rates, pillars):
    if len(list(s)) > 1:
        print    
        
    print s, hz, p    

## Copula

In [ ]:
from maths.copula.marshallolkin import StepWiseIntensitiesMarshallOlkinCopula

copula = StepWiseIntensitiesMarshallOlkinCopula(subsets, hazard_rates, pillars)

In [ ]:
# The last group contains all
# the obligors ids (as a senior CDO tranch)

bc_ids = list(raw_groups[-1]['ids'])
print bc_ids

In [ ]:
bc_subsets_indexes = []
for id_ in bc_ids:
    bc_subsets_indexes += copula.get_indexes_including(id_)
    
bc_subsets_indexes = list(set(bc_subsets_indexes))
bc_subsets_indexes.sort()

print bc_subsets_indexes

## Default times

In [ ]:
default_times_mat = copula.generate_default_times(subsets_indexes=bc_subsets_indexes, number=N)

In [ ]:
default_times_mat_shifted = v_time_offseter(default_times_mat, time_grid)

In [ ]:
index = time_grid.searchsorted(default_times_mat_shifted[:, 0].min())

print time_grid[index-1], time_grid[index-1] + step, time_grid[index]
print default_times_mat[:, 0].min()
print default_times_mat_shifted[:, 0].min()

# Saving

In [ ]:
to_save = {}

to_save["N"] = N
to_save["randomization"] = {
    "distrib": {"loc": 0., "scale": mean},
    "zetas": zetas,
    "maturity": maturity,
}
to_save["credit"] = {
    "copula": copula,
    "bc_subsets_indexes": bc_subsets_indexes,
    "bc_ids": bc_ids,
    "default_times": default_times_mat_shifted,
}
to_save["underlyings"] = underlyings

In [ ]:
import dill
import itertools
import os

for i in itertools.count():
    path, ext = 'data', '.pkl'
    insert = str(i)
    if not i:
        insert = ''
        
    path = "%s%s%s" % (path, insert, ext)
        
    if not os.path.isfile(path):
        with open(path, 'wb') as f:
            dill.dump(to_save, f)
        break